Used Selenium to scrape Goodreads website for the information about the list of books made into movies. This became the basis of comparison for the movies/books database.

In [87]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time
import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
import pickle

In [88]:
driver = webdriver.Chrome(chromedriver)
url = "https://www.goodreads.com/shelf/show/books-made-into-movies?page=1"
driver.get(url)
headers = requests.utils.default_headers()
headers.update(
    {
        'User-Agent': 'Chrome/73.0.3683.103',
    }
)

In [89]:
sign_in_button = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/header/div[1]/div/ul/li[1]/a')
sign_in_button.click()

In [90]:
# See notes about environment variables
%load_ext dotenv
%dotenv
import os
EMAIL = os.environ.get('USERNAME')
PASSWORD = os.environ.get('PASSWORD')

# Show that this is working. Don't do this for PASSWORD!
print(EMAIL)

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
auroraborealis666@gmail.com


In [91]:
username_form = driver.find_element_by_id("user_email")
username_form.send_keys(EMAIL) 

In [92]:
username_form.send_keys(Keys.RETURN)

In [93]:
password_form=driver.find_element_by_id("user_password") # note another approach
password_form.send_keys(PASSWORD) # enter password


In [94]:
password_form.send_keys(Keys.RETURN)

In [96]:

titles = []
ratings = []
number_ratings = []
years_published = []

running = True
timeout = 5

while running:
    try:
        element_present = EC.presence_of_element_located((By.LINK_TEXT, 'next »'))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    bookElements = driver.find_elements_by_css_selector(".elementList > .left")
    for element in bookElements:
        
        link = element.find_element_by_class_name("bookTitle")
        
        title = link.text
        title = title.split("(")[0].strip()
        titles.append(title)
        
        popularity = element.find_element_by_xpath("*[contains(text(), 'avg rating')]").text
        popularity = popularity.split("—")

        rating = popularity[0].strip()
        rating = rating[len("avg rating "):]
        ratings.append(rating)

        number_rating = popularity[1].strip()
        number_rating = number_rating.split(" ")[0]
        number_ratings.append(number_rating)

        year_published = popularity[2].strip()
        parts = year_published.split(" ")
        if (len(parts) < 2):
            years_published.append(float("NaN"))
        else:
            year_published = year_published.split(" ")[1]
            years_published.append(year_published)
    
    try:
        next_button = driver.find_element_by_link_text('next »')
        next_button.click()
    except NoSuchElementException as exception:
        running = False

print(len(titles))

Timed out waiting for page to load
1191


In [103]:
with open('title.pkl', 'wb') as f:
    pickle.dump(titles, f)

In [104]:
with open('rating.pkl', 'wb') as f:
    pickle.dump(ratings, f)

In [105]:
with open('number_rating.pkl', 'wb') as f:
    pickle.dump(number_ratings, f)

In [106]:
with open('year_published.pkl', 'wb') as f:
    pickle.dump(years_published, f)